# 2. Basic Regression with Tensorflow

In [10]:
import tensorflow as tf
from sklearn import datasets # we need this to import the Iris Dataset
import numpy as np

## 2.1 Creating Our Model

### Defining Hyperparameters

In [11]:
learning_rate = 0.02
iterations = 2000

### Defining Placeholders:

These are tensors but without any values. We are telling TF to create tensors with the promise that we will later fill in values for these. This is good if our training data isn't always the same, for example when we are using batches.

In [12]:
x = tf.placeholder(tf.float32,shape=[None,1], name="input-data")
y = tf.placeholder(tf.float32, shape=[None,1], name="ground-truth-labels")

### Defining Variables
Tensorflow knows that it can adjust the weights of these

In [13]:
W = tf.Variable(np.random.randn(), name="weight")
b = tf.Variable(np.random.randn(), name="bias")


### Defining the model
We are using classic OLS regression so our model has the form: 

    y_hat = ß0 + ß1 * X1

In [14]:
y_pred = tf.add(tf.multiply(x,W), b)

### Defining our Loss
We are using the Mean Squared Error (MSE) as out loss / error function

In [15]:
loss = tf.reduce_sum(tf.pow(y_pred - y, 2)) / (2 * 100)

### The optimizer
The optimizer is a TF object that takes care of of adjusting our model. It calculates the gradients and adjusts our weights.

In [16]:
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)


## 2.2 Loading the Data

In [17]:
data = datasets.load_iris()
features = data["data"][:100,0].reshape(100,1)
labels = data["target"][:100].reshape(100,1)
s = np.arange(features.shape[0])
np.random.shuffle(s)
features = features[s]
labels = labels[s]

A helper function to calculate the accuracy of our model

In [50]:
def calc_accuracy(reg_vals, true_vals, limit):
    reg_vals = np.array(reg_vals)
    preds = np.where(reg_vals > limit,1,0)
    acc = (len(true_vals) - np.abs(preds - true_vals).sum()) / len(true_vals)
    return(acc)

## 2.3 Running the model

In [51]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for i in range(iterations):

        # we run the session. Note how the feed_dict variable passes on
        # the variables for our placeholder tensors
        loss_train, _ = sess.run([loss, optimizer], feed_dict={x: features, y: labels})
        if i % 200 == 0:
            print(f"Iteration {i}, Loss: {loss_train}")

    loss_test = sess.run(loss, feed_dict={x:features, y:labels})
    print(f"Our final loss is {loss_test}.")

    y_preds_final = sess.run([y_pred], feed_dict={x:features, y:labels})

Iteration 0, Loss: 5.642588138580322
Iteration 200, Loss: 0.1409493386745453
Iteration 400, Loss: 0.1281639188528061
Iteration 600, Loss: 0.11386275291442871
Iteration 800, Loss: 0.10004881024360657
Iteration 1000, Loss: 0.0879577025771141
Iteration 1200, Loss: 0.07820592075586319
Iteration 1400, Loss: 0.07091391086578369
Iteration 1600, Loss: 0.06585236638784409
Iteration 1800, Loss: 0.06259641796350479
Our final loss is 0.0606623999774456.


### Calculating the final Accuracy:

In [52]:
calc_accuracy(y_preds_final, labels, 0.5)

0.89